In [83]:
# For Sam

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

loading the dataset

In [85]:
dataset = pd.read_csv("Train.csv")
finaltest = pd.read_csv("Test.csv")
riders = pd.read_csv('Riders.csv')

Checking it out

In [86]:
dataset.head(2)

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,...,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,...,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993


In [87]:
riders.head(2)

,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Rider_Id_396,2946,2298,14.0,1159
1,Rider_Id_479,360,951,13.5,176


In [88]:
finaltest.head(2)

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Pickup - Weekday (Mo = 1),Pickup - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,4:44:10 PM,27,3,...,3,5:06:47 PM,8,NaN,NaN,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,5,12:57:35 PM,17,5,...,5,1:25:37 PM,5,NaN,NaN,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868


In [89]:
set(dataset.columns) - set(finaltest.columns)

{'Arrival at Destination - Day of Month',
 'Arrival at Destination - Time',
 'Arrival at Destination - Weekday (Mo = 1)',
 'Time from Pickup to Arrival'}

we must drop the above before moving on

are the columns in the final that are not on training?

In [90]:
set(finaltest.columns) - set(dataset.columns)

set()

In [91]:
xdata = dataset.drop(columns=['Time from Pickup to Arrival', 'User Id', 'Order No', 
'Arrival at Destination - Day of Month',
'Arrival at Destination - Time',
'Arrival at Destination - Weekday (Mo = 1)'])
X = xdata.copy() # I want to run this column everytime I want to restart after changes
X = X.merge(riders, how='left', on=['Rider Id']).drop(columns=['Rider Id'])

f_testx = finaltest.merge(riders, how='left', on=['Rider Id']).drop(columns=['User Id', 'Order No', 'Rider Id'])

In [92]:
y = dataset['Time from Pickup to Arrival'] # The target

Confirm columns if columns on train are those on test

In [93]:
set(f_testx.columns) == set(X.columns)

True

We Split

In [94]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Now things are about to get ugly

In [95]:
### START FUNCTION
def cleaner(input_df, nullthreshold=0.9, correlation_thresh=0.95, day_of_month_cols=[], day_of_week_cols=[]):
    input_dfc = input_df.copy()
    

    #########################################################################################
    # The Code below drops columns that have null values exceeding threshold and Columns that have ONLY one value
    for col in input_df.columns:
        if (sum(input_df[col].isnull())/len(input_df[col]) > nullthreshold) or (len(input_df[col].unique()) == 1):
            input_dfc.drop(columns=[col], inplace=True) 
            
    #########################################################################################

    #########################################################################################
    #This code converts time given by am and pm into seconds then applies cosine and sine
    def time_to_seconds(input_df):
        input_dfc = input_df.copy()

        from datetime import datetime

        for time_col in [col for col in input_df.columns if 'Time' in [col[-4:]]]:

            input_dfc[time_col + '_sin(seconds)'] = \
            input_df[time_col].apply(lambda time: np.sin(
                (datetime.strptime(time, '%I:%M:%S %p') - datetime(1900,1,1)).total_seconds() \
                * (2.*np.pi/86400) )) # there are 86400 seconds in a day

            input_dfc[time_col + '_cos(seconds)'] = \
            input_df[time_col].apply(lambda time: np.cos(
                (datetime.strptime(time, '%I:%M:%S %p') - datetime(1900,1,1)).total_seconds() \
                * (2.*np.pi/86400) ))

            input_dfc.drop(columns=[time_col], inplace=True)

        return input_dfc

    input_dfc2 = time_to_seconds(input_dfc)
    #########################################################################################
    
    #########################################################################################
    # This code encodes ['Platform Type', 'Personal or Business']
    
    def one_encoder(input_df, columns):
    
        return pd.get_dummies(input_df, drop_first=True, columns=columns, dtype=float)

    
    input_dfc2 = one_encoder(input_dfc2, ['Platform Type', 'Personal or Business'])
    #########################################################################################
    
    #########################################################################################
    # This code will remove one of a pair of variables that are 95% correlated
    def correlation_drop(df, thresh):
        while True:
            corr_matrix = df.corr(method = "spearman").abs()
            upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))
            to_drop = [column for column in upper.columns if any(upper[column] > thresh)]
            if len(to_drop) == 0:
                break
            else:
                df = df.drop(to_drop, axis = 1)
            
        return df
    
    input_dfc2 = correlation_drop(input_dfc2, correlation_thresh)
    #########################################################################################

    print(f"Total of {len([x for x in input_df.columns if x not in input_dfc2])} original columns dropped \n")  
    print(f"Total of {len([x for x in input_dfc2.columns if x not in input_df])} new CLEAN columns formed \n")
    print(f"Dataframe now has {len(input_dfc2.columns)} from {len(input_df.columns)} input columns")
    #print([x for x in input_df.columns if x not in input_dfc2])

    return input_dfc2

### END FUNCTION

In [131]:
X1.head()

,Placement - Day of Month,Placement - Weekday (Mo = 1),Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Placement - Time_sin(seconds),Placement - Time_cos(seconds),Platform Type_2,Platform Type_3,Platform Type_4,Personal or Business_Personal
0,9,5,4,20.4,-1.317755,36.830370,-1.300406,36.829741,1637,1309,13.8,549,0.588609,-0.808418,0.0,1.0,0.0,0.0
1,12,5,16,26.4,-1.351453,36.899315,-1.295004,36.814358,396,339,13.6,69,0.189667,-0.981849,0.0,1.0,0.0,1.0
2,30,2,3,NaN,-1.308284,36.843419,-1.300921,36.828195,1023,242,12.5,114,-0.171141,-0.985247,0.0,1.0,0.0,0.0
3,15,5,9,19.2,-1.281301,36.832396,-1.257147,36.795063,886,283,14.5,113,0.623993,-0.781430,0.0,1.0,0.0,0.0
4,13,1,9,15.4,-1.266597,36.792118,-1.295041,36.809817,2311,872,14.1,533,0.517654,-0.855590,0.0,0.0,0.0,1.0


Now we apply our ultimate cleaner

In [122]:
day_of_month_cols = [x for x in X.columns if x[-5:] == 'Month']
day_of_week_cols = [x for x in X.columns if x[-(len('(Mo = 1)')):] == '(Mo = 1)']
X_train = cleaner(X_train, day_of_month_cols=day_of_month_cols, day_of_week_cols=day_of_week_cols)

KeyError: "None of [Index(['Platform Type', 'Personal or Business'], dtype='object')] are in the [columns]"

In [123]:
X_test = cleaner(X_test, day_of_month_cols=day_of_month_cols, day_of_week_cols=day_of_week_cols)

KeyError: "None of [Index(['Platform Type', 'Personal or Business'], dtype='object')] are in the [columns]"

In [99]:
X1 = cleaner(X, day_of_month_cols=day_of_month_cols, day_of_week_cols=day_of_week_cols) # When we want to create a cross validation model

Total of 14 original columns dropped 

Total of 6 new CLEAN columns formed 

Dataframe now has 18 from 26 input columns


We must also apply it on our finial test: 

In [100]:
F_test = cleaner(f_testx, day_of_month_cols=day_of_month_cols, day_of_week_cols=day_of_week_cols)

Total of 14 original columns dropped 

Total of 6 new CLEAN columns formed 

Dataframe now has 18 from 26 input columns


In [101]:
# corr_matrix = X_train.corr(method = "spearman").abs()
# sns.set(font_scale = 1.0)
# f, ax = plt.subplots(figsize=(11, 9))
# sns.heatmap(corr_matrix, cmap= "YlGnBu", square=True, ax = ax)
# f.tight_layout()


Our data is ready

We will standardize the columns which have a max absolute value that is greater than 1

In [102]:
# 'We use this code to apply standard scaler, and still keep our data as a dataframe instead of an array'

# from sklearn_pandas import DataFrameMapper
# from sklearn.preprocessing import StandardScaler

# mapper = DataFrameMapper([(X_train.columns, StandardScaler())])
# scaled_features = mapper.fit_transform(X_train.copy(), 4)
# scaled_features_df = pd.DataFrame(scaled_features, index=X_train.index, columns=X_train.columns)
# scaled_features_df.head(2)

In [103]:
X_train.head(2)

,Placement - Day of Month,Placement - Weekday (Mo = 1),Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Placement - Time_sin(seconds),Placement - Time_cos(seconds),Platform Type_2,Platform Type_3,Platform Type_4,Personal or Business_Personal
19798,7,4,16,16.8,-1.304033,36.784869,-1.229720,36.874551,721,984,13.8,161,-0.194591,-0.980884,0.0,1.0,0.0,0.0
9959,28,5,10,25.7,-1.304524,36.775371,-1.255189,36.782203,532,740,14.0,44,-0.329210,-0.944257,0.0,0.0,0.0,0.0


In [156]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
polinomial_features = PolynomialFeatures(1)
# Construct the pipeline with a standard scaler and a small neural network
estimators = []
estimators.append(('imputer', SimpleImputer(strategy='median')))
estimators.append(('standardize', StandardScaler()))
estimators.append(('plf', polinomial_features))
estimators.append(('mod', Lasso(alpha=0.01)))
model = Pipeline(estimators)

# To begin, let's use only these two features to predict 'cnt' (bicycle count)


# We'll use 5-fold cross validation. That is, a random 80% of the data will be used
# to train the model, and the prediction score will be computed on the remaining 20%.
# This process is repeated five times such that the training sets in each "fold"
# are mutually orthogonal.
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=5)

results = cross_val_score(model, X1, y, cv=kfold, scoring='neg_mean_squared_error')
print ('CV Scoring Result: mean=',np.sqrt(abs(np.mean(results))),'std=',np.std(results))

CV Scoring Result: mean= 796.969038169289 std= 34500.51937564535


In [157]:
estimators

[('imputer',
  SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                missing_values=nan, strategy='median', verbose=0)),
 ('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('plf',
  PolynomialFeatures(degree=1, include_bias=True, interaction_only=False,
                     order='C')),
 ('mod',
  Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
        normalize=False, positive=False, precompute=False, random_state=None,
        selection='cyclic', tol=0.0001, warm_start=False))]

In [104]:
# corr_matrix = scaled_features_df.corr(method = "spearman").abs()
# sns.set(font_scale = 1.0)
# f, ax = plt.subplots(figsize=(11, 9))
# sns.heatmap(corr_matrix, cmap= "YlGnBu", square=True, ax = ax)
# f.tight_layout()


In [105]:
# sfs.fit(scaled_features_df, y_train)
# sfs.k_feature_names_ 
# don't mind this code
# sfs.scorer(LinearRegression(), scaled_features_df, y_train)

In [106]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error

In [107]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

In [136]:
models = [LinearRegression()]
polinomial_features = PolynomialFeatures(2)


for model in models:
    # numeric_transformer = Pipeline(steps=[
    # ('imputer', SimpleImputer(strategy='median')),
    # ('scaler', StandardScaler())])
    
    # preprocessor = ColumnTransformer(
    # transformers=[
    #     ('num', numeric_transformer, cols_to_standard)])
    
    steps = [('imputer', SimpleImputer(strategy='mean')), ('polf', polinomial_features), 
          ('Scaler', StandardScaler()),
          ('model', model)]
    
    pipe = Pipeline(steps)
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    print(model)
    print("model r2 score: %.3f" % pipe.score(X_test, y_test))
    print(f"model rmse score: {np.sqrt(mean_squared_error(y_test, y_pred))}")
    print('\n')
    

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
model r2 score: 0.355
model rmse score: 786.4034935748099




TypeError: mean_squared_error() got an unexpected keyword argument 'squared'

In [109]:
sample = pd.read_csv('SampleSubmission.csv')